In [2]:
from multiprocessing import Process
import os


# multiprocessing

def run_proc(name):
    print('Run child process %s (%s)...' % (name, os.getpid()))


if __name__ == '__main__':
    print('Parent process %s.' % os.getpid())
    p = Process(target=run_proc, args=('test',))
    print('Child process will start.')
    p.start()
    p.join()
    print('Child process end.')

Parent process 18612.
Child process will start.
Child process end.


In [ ]:
# pool 进程池
from multiprocessing import Pool
import os, time, random


def long_time_task(name):
    print('Run task %s (%s)...' % (name, os.getpid()))
    start = time.time()
    time.sleep(random.random() * 3)
    end = time.time()
    print('Task %s runs %0.2f seconds.' % (name, (end - start)))


if __name__ == '__main__':
    print('Parent process %s.' % os.getpid())
    p = Pool(4)
    for i in range(5):
        # 执行异步操作
        p.apply_async(long_time_task, args=(i,))
    print('Waiting for all subprocesses done...')
    p.close()
    p.join()
    print('All subprocesses done.')

Parent process 18612.
Waiting for all subprocesses done...


In [ ]:
#子进程
# 子进程有时候不是自身，而是当作一个外部进程。我们有时候创建子进程候，还要控制子进程的输入和输出
import subprocess  # 启动一个子进程，进行控制输入输出

print('$ nslookup www.zqywuku.top')
r = subprocess.call(['nslookup', 'www.zqywuku.top'])
print('Exit code:', r)

import subprocess

print('$ nslookup')
#subprocess.PIPE 为输入输出错误 提供一个管道，可以进行输入和读取
p = subprocess.Popen(['nslookup'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
output, err = p.communicate(b'set q=mx\npython.org\nexit\n')
print(output.decode('gbk'))
print('Exit code:', p.returncode)


In [ ]:
#进程通信
from multiprocessing import Queue, Process
import os, time, random


def write(q: Queue):
    print('Process to write %s' % os.getpid())
    for value in ['A', 'B', 'C']:
        print('put %s to queue...' % value)
        q.put(value)
        time.sleep(random.random())


def read(q: Queue):
    print('Process to get %s' % os.getpid())
    while True:
        # block = True,是否阻塞直到有参数出现
        value = q.get(True)
        print('Get %s from queue' % value)


if __name__ == '__main__':
    q = Queue()
    pw = Process(target=write, args=(q,))
    pr = Process(target=read, args=(q,))
    pw.start()
    pr.start()
    pw.join()
    # 死循环，强制暂停
    pr.terminate()

# 多线程
**_thread/threading**

In [ ]:
import time, threading


# 新线程执行的代码:
def loop():
    print('thread %s is running...' % threading.current_thread().name)
    n = 0
    while n < 5:
        n = n + 1
        print('thread %s >>> %s' % (threading.current_thread().name, n))
        time.sleep(1)
    print('thread %s ended.' % threading.current_thread().name)


print('thread %s is running...' % threading.current_thread().name)  # 主线程名字：MainThread
t = threading.Thread(target=loop, name='LoopThread')  # 子线程名称：LoopThread
t.start()
t.join()
print('thread %s ended.' % threading.current_thread().name)

In [ ]:
# 多进程，每个进程维护自己的变量，不会影响其他进程的
# 多线程在同一个进程下，所以数据是共享的，操作不当会引起数据不按照预期执行
import time, threading

# 假定这是你的银行存款:
balance = 0

"""加锁"""
lock = threading.Lock()


def change_it(n):
    # 先存后取，结果应该为0:
    global balance
    balance = balance + n
    balance = balance - n


def run_thread(n):
    for i in range(2000000):
        # 先获取锁
        lock.acquire()
        try:
            change_it(n)
        finally:
            # 最后记得释放
            lock.release()


# 1.解释将整个方法的构建都给线程进行操作，不能保证方法的完整性
t1 = threading.Thread(target=run_thread, args=(5,))
t2 = threading.Thread(target=run_thread, args=(8,))

# 2.这种写法，相当于把整个方法作为一个整体交给线程进行操作，所以+-操作都在一起
# t1 = threading.Thread(target=run_thread(5))
# t2 = threading.Thread(target=run_thread(8))
# balance = 0
t1.start()
t2.start()
t1.join()
t2.join()
print(balance)  #存了又取，预期为0；但是多线程会导致操作顺序不一致
"""
解决方案：使用lock锁
"""


# ThreadLocal 全局对象

In [ ]:
import threading

local_school = threading.local()


def process_student():
    # 获取当前线程关联的student:
    std = local_school.student
    print('Hello, %s (in %s)' % (std, threading.current_thread().name))


def process_thread(name):
    # 绑定ThreadLocal的student:
    local_school.student = name
    process_student()


t1 = threading.Thread(target=process_thread, args=('Alice',), name='Thread-A')
t2 = threading.Thread(target=process_thread, args=('Bob',), name='Thread-B')
t1.start()
t2.start()
t1.join()
t2.join()


# 进程 vs 线程
多进程：
    优点:
        - 稳定性高，一个进程崩溃不会影响主进程和其他子进程
    缺点:
        - 创建进程的代价较大，并且受到内存和CPU的限制
多线程:
    优点：
        - 创建较为方便
    缺点：
        - 受其他线程影响，任何一个线程挂掉都会导致整个进程崩溃，因为线程共享进程的内存
# 线程切换
先切切换会有资源的浪费，虽然快但是也要浪费时间。当任务过多，那么就会出现假死的状态。
# 计算密集型和IO密集型
计算密集型：进行大量计算，消耗CPU资源，因为任务过多那么任务切换时间就会越多，CPU执行效率就越低，所以最好计算密集型任务进行的数量等于CPU的核心数；python不适合，建议**c语言**

IO密集型：CPU消耗少，大部分时间等待IO操作完成。所以任务越多，CPU效率越高，减少任务的等待时间。对于IO密集型任务，最合适的语言就是开发效率最高（代码量最少）的语言，脚本语言是首选，C语言最差。
`备注：进程是最小调度单元，线程是最小执行单元，协程基于事件驱动`

# 分布式进程 
managers模块支持多进程分布不同机器上。